In [ ]:
import os
import pymongo
import openai
from llama_index.core import VectorStoreIndex,StorageContext, SimpleDirectoryReader
from llama_index.core.schema import Document
from llama_index.vector_stores.mongodb import MongoDBAtlasVectorSearch
from llama_index.readers.mongodb import SimpleMongoReader

In [ ]:
reader = SimpleMongoReader(uri=os.getenv("MONGODB_URI"))
documents = reader.load_data(
    os.getenv("MONGODB_DB"),
    os.getenv("MONGODB_COLL"), 
    field_names=["title","plot"], # these is a list of the top-level fields in your objects that will be indexed
    query_dict={} # this is a mongo query dict that will filter your data if you don't want to index everything
)
print(documents[0])

In [ ]:
# Create a new client and connect to the server
client = pymongo.MongoClient(os.getenv("MONGODB_URI"))

# create Atlas as a vector store
store = MongoDBAtlasVectorSearch(
    client,
    db_name=os.getenv('MONGODB_DB'),
    collection_name=os.getenv('MONGODB_VECTOR_COLL_LLAMAINDEX'), # this is where your embeddings will be stored
    index_name=os.getenv('MONGODB_VECTOR_INDEX') # this is the name of the index you will need to create
)

# now create an index from all the Documents and store them in Atlas
storage_context = StorageContext.from_defaults(vector_store=store)
index = VectorStoreIndex.from_documents(
    documents, storage_context=storage_context,
    show_progress=True, # this will show you a progress bar as the embeddings are created
)


In [ ]:
prompt="Young Ones Set in the future when water is hard to find, a teenage boy sets out to protect his family and survive"
response = index.as_query_engine(similarity_top_k=5).query(prompt)

print(response)